# APPLICATION TEMPLATE INSITU GRIDDED - SPATIALLY WEIGHTED STATISTICS

## Import libraries

In [ ]:
import warnings

import cads_toolbox
from c3s_eqc_automatic_quality_control import diagnostics, download, plot

## Global settings

In [ ]:
warnings.filterwarnings("ignore")
cads_toolbox.config.USE_CACHE = True

## Set year and statistics to download

In [ ]:
years = list(range(2000, 2010 + 1))
statistics = [
    "maximum",
    "mean",
    "minimum",
]

## Request dictionary

In [ ]:
collection_id = "insitu-gridded-observations-global-and-regional"
request = {
    "format": "zip",  # Added support for single-file zip archives
    "variable": "temperature",
    "statistic": statistics,
    "horizontal_aggregation": "2_5_x_2_5",
    "year": years,
    "version": "v4.03",
    "origin": "cru",
    "region": "global",
    "time_aggregation": "monthly",
}

## Download and compute spatially weighted mean

In [ ]:
ds_mean = download.download_and_transform(
    collection_id,
    request,
    split_all=True,  # Split in multiple single-file requests
    open_with="xarray",
    func=diagnostics.spatial_weighted_mean,
)

## Compute spatially weighted std using cached files

In [ ]:
ds_std = download.download_and_transform(
    collection_id,
    request,
    split_all=1,
    open_with="xarray",
    func=diagnostics.spatial_weighted_std,  # Added another example of diagnostics
)

## Plot statistics

In [ ]:
fig = plot.shaded_std(
    ds_mean, ds_std, ["tasmin", "tasmax", "tas"], "Spatially-weighted fields"
)
fig.show()
fig.write_image(f"{collection_id}_statistics_{years[0]}-{years[-1]}.png")